In [27]:
data = bytearray(open("./xemjp", "rb").read())

In [ ]:
data[:10]

In [ ]:
def pbyte(b):
    return hex(int(b))

In [24]:
hex(12974662)

'0xc5fa46'

In [44]:
# Xemnas? 
start_byte = 0x00c5f75E
end_byte = 0x00c5fa46
# zexion
# start_byte = 0x00c83b61
# end_byte = 0x00c842f4 
subdata = data[start_byte:end_byte]
subdata

bytearray(b'dir\x00warp_building_front\x00atk_dark\x00\x00leave\x00check_atk_dark_hit\x00\x00dead\x00\x00faint_short\x00comeback\x00\x00move_shield\x00move_counter\x00\x00jump start\x00\x00atk_counter\x00jump end\x00\x00idle_middle\x00idle\x00\x00warp_target_middle\x00\x0000\x00\x00warp_target_far\x00rc_invitation_to_dark\x00btl_attack\x00\x00atk_combo\x00warp_target_near\x00\x00btl_short\x00btl_long\x00\x0010\x00\x00atk_shield_short\x00\x00mode_check\x00\x00rvg_short\x00mode_dark\x00rvg_long\x00\x0020\x00\x00mode_super_hard\x00mode_battle\x00mode_hard\x00mode_normal\x00freeze\x00\x00btl_normal\x00\x0030\x00\x00mode_revenge\x00\x00btl_hard\x00\x00atk_shield\x00\x00event\x00btl_super_hard\x00\x00appear\x00\x00idle_time\x00faint_long\x00\x00footwork\x00\x00rvg_normal\x00\x00mode_battle_boss\x00\x00rvg_hard\x00\x00mode_revenge_boss\x00rvg_super_hard\x00\x00idle_short\x00\x00idle_wait\x00rst_normal\x00\x00warp_target_origin\x00\x00rst_hard\x00\x00revenge\x00rst_super_hard\x00\x00scale\x00r

In [53]:
commands = []
command = {"startaddr": 0, "content": bytearray([])}
for i in range(len(subdata)):
    b = subdata[i]
    if b == 0:
        if len(command["content"]) > 0:
            command["endaddr"] = i-1
            commands.append(command)
        command = {"startaddr": i, "content": bytearray([])}
    else:
        command["content"].append(b)

In [142]:
fn = "xemnas.yaml"
import yaml
yaml.dump([{"command": x["content"].decode('utf-8'), "enabled": True, "startaddr": x["startaddr"], "endaddr": x["endaddr"]} for x in commands], open(fn,"w"), default_flow_style=False)

In [147]:
fn = "xemnas.yaml"
import yaml
d = yaml.load(open(fn))#, Loader=yaml.FullLoader)

In [91]:
def get_byteword_start(addr):
    return addr - (addr % 4)
def get_byteword_end(addr):
    if addr % 4 == 0:
        return addr
    return addr + (4 - (addr % 4))

In [148]:
codes = {}
for c in range(len(d)):
    c = d[c]
    if not c["enabled"]:
        print("Canceling {}".format(c["command"]))
        header_start = c["startaddr"] + start_byte
        header_byteword_start = get_byteword_start(header_start)
        header_end = start_byte+c["endaddr"] + 1 # I think theres an off by 1 error I don't wanna debug it
        header_byteword_end = get_byteword_end(header_end)
        words = []
        for i in range(header_byteword_start, header_byteword_end, 4):
            word = {"addr": i, "value": bytearray([0,0,0,0])}
            words.append(word)
        ## To really do this right I need to check if I'm canceling out a adjacent word and in that case dont do this
        start_diff = abs(header_start-header_byteword_start)
        for i in range(start_diff):
            words[0]["value"][i] = data[header_byteword_start+i]
        end_diff = abs(header_end-header_byteword_end) - 1 # Again account for the off by 1 error I'm lazy
        for i in range(end_diff):
            words[-1]["value"][3-i] = data[header_byteword_end-i-1] # another off by one thing
        codes[c["command"]] = words
st = ""
for c in codes:
    st += "// Null out {}\n".format(c)
    for line in codes[c]:
        st += "{} {}\n".format(str(hex(line["addr"]))[2:].zfill(8), ''.join([hex(i)[2:].zfill(2) for i in line["value"]][::-1]))
print(st)

Canceling rc_invitation_to_dark
// Null out rc_invitation_to_dark
00c5f828 00726166
00c5f82c 00000000
00c5f830 00000000
00c5f834 00000000
00c5f838 00000000
00c5f83c 00000000
00c5f840 74620000



In [149]:
header_start

12974123

In [138]:
hex(data[header_byteword_end-0])

'0x6b'

In [120]:
i

0

In [ ]:
# //warp building front
# 00c62f60 00000072
# 00c62f64 00000000
# 00c62f68 00000000
# 00c62f6c 00000000
# 00c62f70 00000000
# 00c62f74 74610000
# // rc invitation to dark
# 00c6302c 00000000
# 00c63030 00000000
# 00c63034 00000000
# 00c63038 00000000
# 00c6303c 00000000


In [ ]:
# quick function to give out stats on the amount of 00 bytes in the file

In [ ]:
# input start/stop address of headers
# parse into list, using 00 byte as separator
# translate names to english, output a json file with true/false

# when given a headers list, generate codes to null out all the falses, and write to a pnach
